In [1]:
from keras.models import Sequential

/usr/local/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [2]:
import numpy as np
import warnings
warnings.filterwarnings('ignore')
import pandas as pd
from sklearn.cross_validation import KFold
from sklearn.neighbors import KernelDensity
from sklearn import svm

/usr/local/lib/python3.6/site-packages/sklearn/cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [3]:
train = pd.read_csv('yelp_data_train.dat', header=0, \
                    delimiter="\t")


In [4]:
train.shape

(2908, 1)

In [5]:
#train

In [6]:
 train.columns.values

array(['date;reviewID;reviewerID;reviewContent;rating;usefulCount;coolCount;funnyCount;fake;hotelID'],
      dtype=object)

In [7]:
train_array = np.array(train)

In [8]:
var=train_array[0][0].split(';')

In [9]:
var = np.array(var)
    

In [10]:
var

array(['9/16/2010', 'Ol', 'nf3q2h-kSQoZK2jBY92FOg',
       '"If you are considering staying here, watch this first: http://www.youtube.com/w…"',
       '1', '8', '2', '6', 'N', 'tQfLGoolUMu2J0igcWcoZg'], dtype='<U82')

In [11]:
rev = []
for i in range(2908):
    var=train_array[i][0].split(';')
    var=np.array(var)
    #rev.append(var[3])
    rev.append(var[3])
    

In [12]:
rev = np.array(rev)

In [13]:
rev.shape

(2908,)

In [14]:
from bs4 import BeautifulSoup  
import re
from nltk.corpus import stopwords
def review_to_words( raw_review ):
    # Function to convert a raw review to a string of words
    # The input is a single string (a raw movie review), and 
    # the output is a single string (a preprocessed movie review)
    #
    # 1. Remove HTML
    review_text = BeautifulSoup(raw_review).get_text() 
    #
    # 2. Remove non-letters        
    letters_only = re.sub("[^a-zA-Z]", " ", review_text) 
    #
    # 3. Convert to lower case, split into individual words
    words = letters_only.lower().split()                             
    #
    # 4. In Python, searching a set is much faster than searching
    #   a list, so convert the stop words to a set
    stops = set(stopwords.words("english"))                  
    # 
    # 5. Remove stop words
    meaningful_words = [w for w in words if not w in stops]   
    #
    # 6. Join the words back into one string separated by space, 
    # and return the result.
    return( " ".join( meaningful_words )) 

In [15]:
# Get the number of reviews based on the dataframe column size

# Initialize an empty list to hold the clean reviews
clean_train_reviews = []

# Loop over each review; create an index i that goes from 0 to the length
# of the movie review list 
for i in range(2908):
    # Call our function for each one, and add the result to the list of
    # clean reviews
    clean_train_reviews.append( review_to_words( rev[i] ) )

In [16]:
print ("Creating the bag of words...\n")
from sklearn.feature_extraction.text import CountVectorizer

# Initialize the "CountVectorizer" object, which is scikit-learn's
# bag of words tool.  
vectorizer = CountVectorizer(analyzer = "word",   \
                             tokenizer = None,    \
                             preprocessor = None, \
                             stop_words = None,   \
                             max_features = 5000) 

# fit_transform() does two functions: First, it fits the model
# and learns the vocabulary; second, it transforms our training data
# into feature vectors. The input to fit_transform should be a list of 
# strings.
train_data_features = vectorizer.fit_transform(clean_train_reviews)

# Numpy arrays are easy to work with, so convert the result to an 
# array
train_data_features = train_data_features.toarray()

Creating the bag of words...



In [17]:
train_data_features.shape

(2908, 5000)

In [18]:
rev2 = []
for i in range(2908):
    var=train_array[i][0].split(';')
    var=np.array(var)
    #rev.append(var[3])
    rev2.append(var[8])

In [19]:
rev2 = np.array(rev2)
rev2.shape

(2908,)

In [20]:
n=0
y=0
zeros=0
ones=0
for i in range(2908):
    if (rev2[i] == 'N'):
        n+=1
    elif (rev2[i] == 'Y'):
        y+=1
    elif (rev2[i] == '0'):
        zeros+=1
    elif (rev2[i] == '1'):
        ones+=1
    #else:
        #print(rev2[i])
    

In [21]:
print ("Training the random forest...")
from sklearn.ensemble import RandomForestClassifier

# Initialize a Random Forest classifier with 100 trees
forest = RandomForestClassifier(n_estimators = 100) 

# Fit the forest to the training set, using the bag of words as 
# features and the sentiment labels as the response variable
#
# This may take a few minutes to run
forest = forest.fit( train_data_features[:2000][:], rev2[:2000] )

Training the random forest...


In [22]:
YPred = forest.predict(train_data_features[2000:][:])

In [23]:
Err=0
for i in range(908):
    if YPred[i] != rev2[i+2000]:
        print(rev2[i+2000])

0
0
0
0
0
 (5) Hotel lobby packed with a kid's convention
2
0
0
0
0
0
0
0
0
0
0
0
9
1
1
0
0
0
0
1
1
0
0
12
0
0
0
0
Y
Y
Y
Y
Y
Y
Y
Y
Y
Y
Y
Y
Y
Y
Y
Y
Y
Y
Y
Y
Y
Y
Y
Y
Y
Y
0
Y
Y
Y
Y
Y
Y
Y
0
Y
Y
Y
Y
Y
Y
Y
Y
Y
Y
Y
Y
Y
Y
Y
Y
Y
Y
0
Y
Y
Y
Y
Y
Y
Y
Y
Y
Y
Y
Y
Y
Y
Y
Y
Y
Y
Y
Y
Y
Y
Y
Y
0
Y
0
Y
Y
Y
Y
Y
Y
Y
Y
Y
Y
Y
Y
Y
Y
Y
Y
Y
0
Y
Y
Y
Y
Y
Y
Y
Y
Y
Y
Y
Y
Y
Y
Y
Y
Y
Y
Y
Y
Y
Y
Y
Y
Y
Y
Y
Y
Y
Y
Y
Y
Y
Y
Y
Y
Y
Y
Y
0
Y
Y
Y
Y
Y
Y
Y
Y
Y
0
Y
Y
Y
Y
Y
Y
Y
Y
Y
Y
Y
Y
Y
Y
Y
Y
Y
0
Y
Y
Y
Y
Y
Y
Y
Y
Y
Y
Y
Y
Y
Y
Y
Y
Y
Y
Y
Y
Y
Y
Y
Y
Y
Y
Y
Y
Y
Y
Y
Y
Y
0
Y
Y
Y
Y
Y
Y
0
Y
Y
Y
Y
Y
Y
Y
Y
Y
Y
Y
Y
Y
Y
Y
Y
Y
Y
Y
Y
Y
0
Y
Y
Y
Y
Y
Y
Y
Y
Y
Y
Y
Y
Y
Y
Y
Y
Y
Y
Y
Y
Y
Y
Y
Y
Y
Y
Y
Y
Y
Y
0
Y
Y
Y
Y
Y
Y
Y
Y
Y
Y
Y
Y
Y
Y
Y
Y
Y
Y
Y
Y
Y
Y
0
Y
Y
Y
Y
Y
Y
Y
Y
Y
Y
Y
Y
Y
Y
Y
Y
Y
Y
Y
Y
Y
Y
Y
Y
Y
Y
Y
Y
Y
Y
Y
Y
Y
Y
Y
Y
Y
Y
Y
Y
Y
Y
Y
Y
Y
Y
Y
Y
Y
Y
Y
Y
Y
Y
Y
Y
Y
Y
Y
Y
Y
Y
Y
Y
Y
Y
Y
Y
Y
Y
Y
Y
Y
Y
0
Y
Y
Y
Y
Y
Y
Y
Y
Y
Y
Y
0
3
0
0
0
0
0
0
0
2
0
11
Y
Y
Y
Y
Y
Y
Y
Y
Y
Y
Y
Y
Y
Y
Y
Y
Y
Y
Y
Y
Y
Y


In [24]:

df=pd.read_csv('yelp_data_train.dat', sep='\t')

In [24]:

df = np.array(df)

In [25]:
str1=str(df[0][0])

In [26]:
str1=str1.split(';')

In [27]:
str1

['9/16/2010',
 'Ol',
 'nf3q2h-kSQoZK2jBY92FOg',
 '"If you are considering staying here, watch this first: http://www.youtube.com/w…"',
 '1',
 '8',
 '2',
 '6',
 'N',
 'tQfLGoolUMu2J0igcWcoZg']

In [28]:
str1[-2:-1]

['N']

In [29]:
str1[3:4]

['"If you are considering staying here, watch this first: http://www.youtube.com/w…"']

In [30]:
rev_test = []
for i in range(2908):
    var=df[i][0].split(';')
    #var=np.array(var)
    #rev.append(var[3])
    rev_test.append(var[-2:-1])

In [31]:
rev_test = np.array(rev_test)

In [32]:
rev_test = rev_test.reshape(2908,)
rev_test.shape

(2908,)

In [33]:
n=0
y=0
zeros=0
ones=0
for i in range(2908):
    if (rev_test[i] == 'N'):
        n+=1
    elif (rev_test[i] == 'Y'):
        y+=1
    elif (rev_test[i] == '0'):
        zeros+=1
    elif (rev_test[i] == '1'):
        ones+=1

In [34]:
n

2516

In [35]:
y

392

In [36]:
rev_train = []
for i in range(2908):
    var=df[i][0].split(';')
    #var=np.array(var)
    #rev.append(var[3])
    rev_train.append(var[3:4])

In [37]:
rev_id_train = []
for i in range(2908):
    var=df[i][0].split(';')
    #var=np.array(var)
    #rev.append(var[3])
    rev_id_train.append(var[2:3])

In [38]:
rev_date_train = []
for i in range(2908):
    var=df[i][0].split(';')
    #var=np.array(var)
    #rev.append(var[3])
    rev_date_train.append(var[0:1])

In [39]:
rev_date_train = np.array(rev_date_train)

In [40]:
rev_train = np.array(rev_train)

In [41]:
rev_test

array(['N', 'N', 'N', ..., 'Y', 'Y', 'Y'], dtype='<U1')

In [42]:
# Get the number of reviews based on the dataframe column size

# Initialize an empty list to hold the clean reviews
clean_train_reviews = []

# Loop over each review; create an index i that goes from 0 to the length
# of the movie review list 
for i in range(2908):
    # Call our function for each one, and add the result to the list of
    # clean reviews
    clean_train_reviews.append( review_to_words( str(rev_train[i]) ) )

In [43]:
print ("Creating the bag of words...\n")
from sklearn.feature_extraction.text import CountVectorizer

# Initialize the "CountVectorizer" object, which is scikit-learn's
# bag of words tool.  
vectorizer = CountVectorizer(analyzer = "word",   \
                             tokenizer = None,    \
                             preprocessor = None, \
                             stop_words = None,   \
                             max_features = 5000) 

# fit_transform() does two functions: First, it fits the model
# and learns the vocabulary; second, it transforms our training data
# into feature vectors. The input to fit_transform should be a list of 
# strings.
train_data_features = vectorizer.fit_transform(clean_train_reviews)

# Numpy arrays are easy to work with, so convert the result to an 
# array
train_data_features = train_data_features.toarray()

Creating the bag of words...



In [44]:
vocab = vectorizer.get_feature_names()
#print (vocab)

# bigram Vectorizer

In [45]:
bigram_vectorizer = CountVectorizer(ngram_range=(1, 2), token_pattern=r'\b\w+\b', min_df=1) 
train_data_features1 = vectorizer.fit_transform(clean_train_reviews)

# Numpy arrays are easy to work with, so convert the result to an 
# array
train_data_features1 = train_data_features1.toarray()

In [46]:
vocab_bi = vectorizer.get_feature_names()
#print (vocab_bi)

In [47]:
train_data_features1.shape

(2908, 5000)

In [48]:
train_data_features.shape

(2908, 5000)

In [49]:
from sklearn.feature_extraction.text import TfidfTransformer
tfidf_transformer = TfidfTransformer(smooth_idf=False)
train_data_features1 = tfidf_transformer.fit_transform(train_data_features1)
train_data_features1

<2908x5000 sparse matrix of type '<class 'numpy.float64'>'
	with 172979 stored elements in Compressed Sparse Row format>

In [50]:
from sklearn.cross_validation import StratifiedKFold
from sklearn.ensemble import RandomForestClassifier
from sklearn import metrics
n_folds = 5
score = 0.0
skf = StratifiedKFold(rev_test, n_folds)

avg_score=0

for train_index, test_index in skf:
    X_train, X_test = train_data_features[train_index], train_data_features[test_index]
    y_train, y_test = rev_test[train_index], rev_test[test_index]
    forest = RandomForestClassifier(n_estimators = 50) 
    forest = forest.fit( X_train, y_train )
    #YPred = forest.predict(X_test)
    score = forest.score(X_test,y_test)
    avg_score += score 
    #y.append(YPred)
    print(score)
    #y_test1=[]
    #for ii in range(y_test.size):
    #    if (y_test[ii] == 'N'):
    #        y_test1.append(0)
    #    else:
    #        y_test1.append(1)
    #YPred1=[]
    #for ii in range(YPred.size):
    #    if (YPred[ii] == 'N'):
    #        YPred1.append(0)
    #    else:
    #        YPred1.append(1)    
    #fpr, tpr, thresholds = metrics.roc_curve(y_test1, YPred1, pos_label=2)
    #print("AUC",metrics.auc(fpr, tpr))
    #print(YPred.shape)
    #Err=0
    #for i in range(YPred.size):
     #   if YPred[i] != y_test[i]:
          #  Err+=1
            #print(YPred[i], y_test[i])
    #print (Err)
print("avg",avg_score/n_folds)

0.8542024013722127
0.8625429553264605
0.8657487091222031
0.8450946643717728
0.8588640275387264
avg 0.857290551546275


In [51]:
from sklearn.naive_bayes import MultinomialNB
n_folds = 5
score = 0.0
skf = StratifiedKFold(rev_test, n_folds)


for train_index, test_index in skf:
    X_train, X_test = train_data_features[train_index], train_data_features[test_index]
    y_train, y_test = rev_test[train_index], rev_test[test_index]
    mnb = MultinomialNB()
    mnb.fit(X_train, y_train)
    #YPred = forest.predict(X_test)
    #y.append(YPred)
    print(mnb.score(X_test,y_test))
    #print(YPred.shape)
    #Err=0
    #for i in range(YPred.size):
     #   if YPred[i] != y_test[i]:
          #  Err+=1
            #print(YPred[i], y_test[i])
    #print (Err)

0.8010291595197255
0.7972508591065293
0.8175559380378657
0.7160068846815835
0.7986230636833046


In [52]:
from sklearn.linear_model import PassiveAggressiveClassifier

In [53]:
n_folds = 5
score = 0.0
skf = StratifiedKFold(rev_test, n_folds)


for train_index, test_index in skf:
    X_train, X_test = train_data_features[train_index], train_data_features[test_index]
    y_train, y_test = rev_test[train_index], rev_test[test_index]
    PAC = PassiveAggressiveClassifier(n_iter=5)
    PAC.fit(X_train, y_train)
    #YPred = forest.predict(X_test)
    #y.append(YPred)
    print(PAC.score(X_test,y_test))
    #print(YPred.shape)
    #Err=0
    #for i in range(YPred.size):
     #   if YPred[i] != y_test[i]:
          #  Err+=1
            #print(YPred[i], y_test[i])
    #print (Err)

/usr/local/lib/python3.6/site-packages/sklearn/linear_model/stochastic_gradient.py:117: DeprecationWarning: n_iter parameter is deprecated in 0.19 and will be removed in 0.21. Use max_iter and tol instead.
  DeprecationWarning)


0.7838765008576329


/usr/local/lib/python3.6/site-packages/sklearn/linear_model/stochastic_gradient.py:117: DeprecationWarning: n_iter parameter is deprecated in 0.19 and will be removed in 0.21. Use max_iter and tol instead.
  DeprecationWarning)


0.788659793814433


/usr/local/lib/python3.6/site-packages/sklearn/linear_model/stochastic_gradient.py:117: DeprecationWarning: n_iter parameter is deprecated in 0.19 and will be removed in 0.21. Use max_iter and tol instead.
  DeprecationWarning)


0.8416523235800344


/usr/local/lib/python3.6/site-packages/sklearn/linear_model/stochastic_gradient.py:117: DeprecationWarning: n_iter parameter is deprecated in 0.19 and will be removed in 0.21. Use max_iter and tol instead.
  DeprecationWarning)


0.8037865748709122
0.7900172117039587


/usr/local/lib/python3.6/site-packages/sklearn/linear_model/stochastic_gradient.py:117: DeprecationWarning: n_iter parameter is deprecated in 0.19 and will be removed in 0.21. Use max_iter and tol instead.
  DeprecationWarning)


# Gaussian Naive Bayes

In [54]:
from sklearn.naive_bayes import GaussianNB
n_folds = 5
score = 0.0
skf = StratifiedKFold(rev_test, n_folds)


for train_index, test_index in skf:
    X_train, X_test = train_data_features[train_index], train_data_features[test_index]
    y_train, y_test = rev_test[train_index], rev_test[test_index]
    gnb = GaussianNB()
    gnb.fit(X_train, y_train)
    #YPred = forest.predict(X_test)
    #y.append(YPred)
    print(gnb.score(X_test,y_test))
    #print(YPred.shape)
    #Err=0
    #for i in range(YPred.size):
     #   if YPred[i] != y_test[i]:
          #  Err+=1
            #print(YPred[i], y_test[i])
    #print (Err)

0.7735849056603774
0.7835051546391752
0.8141135972461274
0.7624784853700516
0.774526678141136


In [55]:
from sklearn.naive_bayes import BernoulliNB
n_folds = 5
score = 0.0
skf = StratifiedKFold(rev_test, n_folds)


for train_index, test_index in skf:
    X_train, X_test = train_data_features[train_index], train_data_features[test_index]
    y_train, y_test = rev_test[train_index], rev_test[test_index]
    bnb = BernoulliNB()
    bnb.fit(X_train, y_train)
    #YPred = forest.predict(X_test)
    #y.append(YPred)
    print(bnb.score(X_test,y_test))

0.8164665523156089
0.7474226804123711
0.8313253012048193
0.7504302925989673
0.8227194492254734


In [56]:
from sklearn import svm
n_folds = 5
score = 0.0
skf = StratifiedKFold(rev_test, n_folds)


for train_index, test_index in skf:
    X_train, X_test = train_data_features[train_index], train_data_features[test_index]
    y_train, y_test = rev_test[train_index], rev_test[test_index]
    
    #YPred = forest.predict(X_test)
    #y.append(YPred)
    print(svm.SVC(kernel='rbf').fit(X_train, y_train).score(X_test,y_test))

0.8644939965694682
0.8642611683848798
0.8657487091222031
0.8657487091222031
0.8657487091222031


In [57]:
from sklearn.neural_network import MLPClassifier
n_folds = 5
score = 0.0
skf = StratifiedKFold(rev_test, n_folds)
clf = MLPClassifier(solver='lbfgs', alpha=1e-5, hidden_layer_sizes=(5, 2), random_state=1)

for train_index, test_index in skf:
    X_train, X_test = train_data_features[train_index], train_data_features[test_index]
    y_train, y_test = rev_test[train_index], rev_test[test_index]
    
    clf.fit(X_train, y_train)
    #YPred = forest.predict(X_test)
    #y.append(YPred)
    print(clf.score(X_test,y_test))

0.7547169811320755
0.7852233676975945
0.8261617900172117
0.7211703958691911
0.7831325301204819


In [58]:
from sklearn.neighbors import KNeighborsClassifier
n_folds = 5
score = 0.0
skf = StratifiedKFold(rev_test, n_folds)
neigh = KNeighborsClassifier(n_neighbors=25)

for train_index, test_index in skf:
    X_train, X_test = train_data_features[train_index], train_data_features[test_index]
    y_train, y_test = rev_test[train_index], rev_test[test_index]
    
    #YPred = forest.predict(X_test)
    #y.append(YPred)
    print(neigh.fit(X_train, y_train).score(X_test,y_test))

0.8644939965694682
0.8625429553264605
0.8657487091222031
0.8640275387263339
0.8640275387263339


In [59]:
from sklearn.linear_model import LogisticRegression

n_folds = 5
score = 0.0
skf = StratifiedKFold(rev_test, n_folds)
logreg = LogisticRegression()

for train_index, test_index in skf:
    X_train, X_test = train_data_features[train_index], train_data_features[test_index]
    y_train, y_test = rev_test[train_index], rev_test[test_index]
    
    #YPred = forest.predict(X_test)
    #y.append(YPred)
    print(logreg.fit(X_train, y_train).score(X_test,y_test))

0.8010291595197255
0.8109965635738832
0.8261617900172117
0.7676419965576592
0.8141135972461274


In [60]:
from sklearn import svm
n_folds = 5
score = 0.0
skf = StratifiedKFold(rev_test, n_folds)


for train_index, test_index in skf:
    X_train, X_test = train_data_features[train_index], train_data_features[test_index]
    y_train, y_test = rev_test[train_index], rev_test[test_index]
    
    #YPred = forest.predict(X_test)
    #y.append(YPred)
    print(svm.SVC(kernel='linear').fit(X_train, y_train).score(X_test,y_test))

0.7324185248713551
0.7525773195876289
0.802065404475043
0.693631669535284
0.7641996557659209


In [61]:
from sklearn import svm
n_folds = 5
score = 0.0
skf = StratifiedKFold(rev_test, n_folds)


for train_index, test_index in skf:
    X_train, X_test = train_data_features[train_index], train_data_features[test_index]
    y_train, y_test = rev_test[train_index], rev_test[test_index]
    
    #YPred = forest.predict(X_test)
    #y.append(YPred)
    print(svm.SVC(kernel='sigmoid').fit(X_train, y_train).score(X_test,y_test))

0.8644939965694682
0.8642611683848798
0.8657487091222031
0.8657487091222031
0.8657487091222031


In [62]:
from sklearn import svm
n_folds = 5
score = 0.0
skf = StratifiedKFold(rev_test, n_folds)


for train_index, test_index in skf:
    X_train, X_test = train_data_features[train_index], train_data_features[test_index]
    y_train, y_test = rev_test[train_index], rev_test[test_index]
    
    #YPred = forest.predict(X_test)
    #y.append(YPred)
    print(svm.LinearSVC().fit(X_train, y_train).score(X_test,y_test))

0.7341337907375644
0.7457044673539519
0.802065404475043
0.7056798623063684
0.7624784853700516


# testing Bigram - start

In [57]:
from sklearn import svm
n_folds = 5
score = 0.0
skf = StratifiedKFold(rev_test, n_folds)


for train_index, test_index in skf:
    X_train, X_test = train_data_features1[train_index], train_data_features1[test_index]
    y_train, y_test = rev_test[train_index], rev_test[test_index]
    
    #YPred = forest.predict(X_test)
    #y.append(YPred)
    print(svm.SVC(kernel='sigmoid').fit(X_train, y_train).score(X_test,y_test))

0.8644939965694682
0.8642611683848798
0.8657487091222031
0.8657487091222031
0.8657487091222031


# testing Bigram - end

# Testing Word Vectors

In [58]:
# Import various modules for string cleaning
from bs4 import BeautifulSoup
import re
from nltk.corpus import stopwords

def review_to_wordlist( review, remove_stopwords=False ):
    # Function to convert a document to a sequence of words,
    # optionally removing stop words.  Returns a list of words.
    #
    # 1. Remove HTML
    review_text = BeautifulSoup(review).get_text()
    #  
    # 2. Remove non-letters
    review_text = re.sub("[^a-zA-Z]"," ", review_text)
    #
    # 3. Convert words to lower case and split them
    words = review_text.lower().split()
    #
    # 4. Optionally remove stop words (false by default)
    if remove_stopwords:
        stops = set(stopwords.words("english"))
        words = [w for w in words if not w in stops]
    #
    # 5. Return a list of words
    return(words)

In [59]:
# Download the punkt tokenizer for sentence splitting
import nltk.data

# Load the punkt tokenizer
tokenizer = nltk.data.load('tokenizers/punkt/english.pickle')

# Define a function to split a review into parsed sentences
def review_to_sentences( review, tokenizer, remove_stopwords=False ):
    # Function to split a review into parsed sentences. Returns a 
    # list of sentences, where each sentence is a list of words
    #
    # 1. Use the NLTK tokenizer to split the paragraph into sentences
    raw_sentences = tokenizer.tokenize(review.strip())
    #
    # 2. Loop over each sentence
    sentences = []
    for raw_sentence in raw_sentences:
        # If a sentence is empty, skip it
        if len(raw_sentence) > 0:
            # Otherwise, call review_to_wordlist to get a list of words
            sentences.append( review_to_wordlist( raw_sentence, \
              remove_stopwords ))
    #
    # Return the list of sentences (each sentence is a list of words,
    # so this returns a list of lists
    return sentences

In [60]:
sentences = []  # Initialize an empty list of sentences

print ("Parsing sentences from training set")
for i in range(2908):
    sentences += review_to_sentences(rev[i], tokenizer)


Parsing sentences from training set


In [61]:
sentences = np.array(sentences)

In [62]:
sentences.shape

(29216,)

In [63]:
# Import the built-in logging module and configure it so that Word2Vec 
# creates nice output messages
import logging
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s',\
    level=logging.INFO)

# Set values for various parameters
num_features = 300    # Word vector dimensionality                      
min_word_count = 40   # Minimum word count                        
num_workers = 4       # Number of threads to run in parallel
context = 10          # Context window size                                                                                    
downsampling = 1e-3   # Downsample setting for frequent words

# Initialize and train the model (this will take some time)
from gensim.models import word2vec
print ("Training model...")
model = word2vec.Word2Vec(sentences, workers=num_workers, \
            size=num_features, min_count = min_word_count, \
            window = context, sample = downsampling)

# If you don't plan to train the model any further, calling 
# init_sims will make the model much more memory-efficient.
model.init_sims(replace=True)

# It can be helpful to create a meaningful model name and 
# save the model for later use. You can load it later using Word2Vec.load()
model_name = "300features_40minwords_10context"
model.save(model_name)

2018-04-03 12:15:10,813 : INFO : 'pattern' package not found; tag filters are not available for English
2018-04-03 12:15:10,819 : INFO : collecting all words and their counts
2018-04-03 12:15:10,819 : INFO : PROGRESS: at sentence #0, processed 0 words, keeping 0 word types
2018-04-03 12:15:10,858 : INFO : PROGRESS: at sentence #10000, processed 148705 words, keeping 8015 word types
2018-04-03 12:15:10,902 : INFO : PROGRESS: at sentence #20000, processed 300514 words, keeping 11567 word types
2018-04-03 12:15:10,946 : INFO : collected 13761 word types from a corpus of 443470 raw words and 29216 sentences
2018-04-03 12:15:10,947 : INFO : Loading a fresh vocabulary
2018-04-03 12:15:10,956 : INFO : min_count=40 retains 1077 unique words (7% of original 13761, drops 12684)
2018-04-03 12:15:10,957 : INFO : min_count=40 leaves 386405 word corpus (87% of original 443470, drops 57065)
2018-04-03 12:15:10,964 : INFO : deleting the raw counts dictionary of 13761 items
2018-04-03 12:15:10,965 : IN

Training model...


2018-04-03 12:15:11,216 : INFO : worker thread finished; awaiting finish of 3 more threads
2018-04-03 12:15:11,217 : INFO : worker thread finished; awaiting finish of 2 more threads
2018-04-03 12:15:11,219 : INFO : worker thread finished; awaiting finish of 1 more threads
2018-04-03 12:15:11,222 : INFO : worker thread finished; awaiting finish of 0 more threads
2018-04-03 12:15:11,223 : INFO : EPOCH - 1 : training on 443470 raw words (253440 effective words) took 0.2s, 1150481 effective words/s
2018-04-03 12:15:11,443 : INFO : worker thread finished; awaiting finish of 3 more threads
2018-04-03 12:15:11,445 : INFO : worker thread finished; awaiting finish of 2 more threads
2018-04-03 12:15:11,448 : INFO : worker thread finished; awaiting finish of 1 more threads
2018-04-03 12:15:11,451 : INFO : worker thread finished; awaiting finish of 0 more threads
2018-04-03 12:15:11,452 : INFO : EPOCH - 2 : training on 443470 raw words (253286 effective words) took 0.2s, 1162531 effective words/s


In [64]:
model

In [65]:
 model.most_similar("door")

[('elevator', 0.9163681864738464),
 ('down', 0.9068641066551208),
 ('into', 0.788445770740509),
 ('ice', 0.7834097146987915),
 ('set', 0.7741348147392273),
 ('cold', 0.7718610763549805),
 ('doors', 0.7655811309814453),
 ('hall', 0.7634575366973877),
 ('open', 0.7625901699066162),
 ('off', 0.7540481090545654)]

In [66]:
from gensim.models import Word2Vec
model = Word2Vec.load("300features_40minwords_10context")

2018-04-03 12:15:17,295 : INFO : loading Word2Vec object from 300features_40minwords_10context
2018-04-03 12:15:17,316 : INFO : loading wv recursively from 300features_40minwords_10context.wv.* with mmap=None
2018-04-03 12:15:17,317 : INFO : setting ignored attribute vectors_norm to None
2018-04-03 12:15:17,318 : INFO : loading vocabulary recursively from 300features_40minwords_10context.vocabulary.* with mmap=None
2018-04-03 12:15:17,318 : INFO : loading trainables recursively from 300features_40minwords_10context.trainables.* with mmap=None
2018-04-03 12:15:17,319 : INFO : setting ignored attribute cum_table to None
2018-04-03 12:15:17,320 : INFO : loaded 300features_40minwords_10context


In [67]:
model.syn0.shape

AttributeError: 'Word2Vec' object has no attribute 'syn0'

In [68]:
def makeFeatureVec(words, model, num_features):
    # Function to average all of the word vectors in a given
    # paragraph
    #
    # Pre-initialize an empty numpy array (for speed)
    featureVec = np.zeros((num_features,),dtype="float32")
    #
    nwords = 0.
    # 
    # Index2word is a list that contains the names of the words in 
    # the model's vocabulary. Convert it to a set, for speed 
    index2word_set = set(model.index2word)
    #
    # Loop over each word in the review and, if it is in the model's
    # vocaublary, add its feature vector to the total
    for word in words:
        if word in index2word_set: 
            nwords = nwords + 1.
            featureVec = np.add(featureVec,model[word])
    # 
    # Divide the result by the number of words to get the average
    featureVec = np.divide(featureVec,nwords)
    return featureVec


In [69]:
def getAvgFeatureVecs(reviews, model, num_features):
    # Given a set of reviews (each one a list of words), calculate 
    # the average feature vector for each one and return a 2D numpy array 
    # 
    # Initialize a counter
    counter = 0.
    # 
    # Preallocate a 2D numpy array, for speed
    reviewFeatureVecs = np.zeros((len(reviews),num_features),dtype="float32")
    # 
    # Loop through the reviews
    for review in reviews:
       
       # Print a status message every 1000th review
           
           
           # Call the function (defined above) that makes average feature vectors
            reviewFeatureVecs[counter] = makeFeatureVec(review, model, \
               num_features)
           
            counter = counter + 1.
    return reviewFeatureVecs

In [70]:
# ****************************************************************
# Calculate average feature vectors for training and testing sets,
# using the functions we defined above. Notice that we now use stop word
# removal.

clean_train_reviews = []
for review in rev_train:
    clean_train_reviews.append( review_to_wordlist( str(review), remove_stopwords=True ))

trainDataVecs = getAvgFeatureVecs( clean_train_reviews, model, num_features )

print ("Creating average feature vecs for test reviews")
clean_test_reviews = []
for review in rev_train:
    clean_test_reviews.append( review_to_wordlist( str(review), \
        remove_stopwords=True ))

testDataVecs = getAvgFeatureVecs( clean_test_reviews, model, num_features )

AttributeError: 'Word2Vec' object has no attribute 'index2word'

In [71]:
trainDataVecs.shape

NameError: name 'trainDataVecs' is not defined

In [72]:
for i in range(2908):
    for j in range(300):
        if(np.isnan(trainDataVecs[i][j])):
            trainDataVecs[i][j]=0
            print(i,j)

NameError: name 'trainDataVecs' is not defined

In [73]:
from sklearn import svm
n_folds = 5
score = 0.0
skf = StratifiedKFold(rev_test, n_folds)


for train_index, test_index in skf:
    X_train, X_test = trainDataVecs[train_index], trainDataVecs[test_index]
    y_train, y_test = rev_test[train_index], rev_test[test_index]
    
    #YPred = forest.predict(X_test)
    #y.append(YPred)
    print(svm.SVC(kernel='rbf').fit(X_train, y_train).score(X_test,y_test))

NameError: name 'trainDataVecs' is not defined

In [ ]:
from sklearn import svm
n_folds = 5
score = 0.0
skf = StratifiedKFold(rev_test, n_folds)


for train_index, test_index in skf:
    X_train, X_test = trainDataVecs[train_index], trainDataVecs[test_index]
    y_train, y_test = rev_test[train_index], rev_test[test_index]
    
    #YPred = forest.predict(X_test)
    #y.append(YPred)
    print(svm.SVC(kernel='linear').fit(X_train, y_train).score(X_test,y_test))

In [ ]:
from sklearn.neighbors import KNeighborsClassifier
n_folds = 5
score = 0.0
skf = StratifiedKFold(rev_test, n_folds)
neigh = KNeighborsClassifier(n_neighbors=35)

for train_index, test_index in skf:
    X_train, X_test = trainDataVecs[train_index], trainDataVecs[test_index]
    y_train, y_test = rev_test[train_index], rev_test[test_index]
    
    #YPred = forest.predict(X_test)
    #y.append(YPred)
    print(neigh.fit(X_train, y_train).score(X_test,y_test))

In [ ]:
from sklearn.naive_bayes import BernoulliNB
n_folds = 5
score = 0.0
skf = StratifiedKFold(rev_test, n_folds)


for train_index, test_index in skf:
    X_train, X_test = trainDataVecs[train_index], trainDataVecs[test_index]
    y_train, y_test = rev_test[train_index], rev_test[test_index]
    bnb = BernoulliNB()
    bnb.fit(X_train, y_train)
    #YPred = forest.predict(X_test)
    #y.append(YPred)
    print(bnb.score(X_test,y_test))

In [ ]:
from sklearn.cross_validation import StratifiedKFold
from sklearn.ensemble import RandomForestClassifier
from sklearn import metrics
n_folds = 5
score = 0.0
skf = StratifiedKFold(rev_test, n_folds)

avg_score=0

for train_index, test_index in skf:
    X_train, X_test = trainDataVecs[train_index], trainDataVecs[test_index]
    y_train, y_test = rev_test[train_index], rev_test[test_index]
    forest = RandomForestClassifier(n_estimators = 80) 
    forest = forest.fit( X_train, y_train )
    #YPred = forest.predict(X_test)
    score = forest.score(X_test,y_test)
    avg_score += score 
    #y.append(YPred)
    print(score)
    #y_test1=[]
    #for ii in range(y_test.size):
    #    if (y_test[ii] == 'N'):
    #        y_test1.append(0)
    #    else:
    #        y_test1.append(1)
    #YPred1=[]
    #for ii in range(YPred.size):
    #    if (YPred[ii] == 'N'):
    #        YPred1.append(0)
    #    else:
    #        YPred1.append(1)    
    #fpr, tpr, thresholds = metrics.roc_curve(y_test1, YPred1, pos_label=2)
    #print("AUC",metrics.auc(fpr, tpr))
    #print(YPred.shape)
    #Err=0
    #for i in range(YPred.size):
     #   if YPred[i] != y_test[i]:
          #  Err+=1
            #print(YPred[i], y_test[i])
    #print (Err)
print("avg",avg_score/n_folds)

In [ ]:
from sklearn import svm
n_folds = 5
score = 0.0
skf = StratifiedKFold(rev_test, n_folds)


for train_index, test_index in skf:
    X_train, X_test = trainDataVecs[train_index], trainDataVecs[test_index]
    y_train, y_test = rev_test[train_index], rev_test[test_index]
    
    #YPred = forest.predict(X_test)
    #y.append(YPred)
    print(svm.LinearSVC().fit(X_train, y_train).score(X_test,y_test))

In [ ]:
from sklearn.linear_model import LogisticRegression

n_folds = 5
score = 0.0
skf = StratifiedKFold(rev_test, n_folds)
logreg = LogisticRegression()

for train_index, test_index in skf:
    X_train, X_test = trainDataVecs[train_index], trainDataVecs[test_index]
    y_train, y_test = rev_test[train_index], rev_test[test_index]
    
    #YPred = forest.predict(X_test)
    #y.append(YPred)
    print(logreg.fit(X_train, y_train).score(X_test,y_test))

# Using Reviewer's Data

In [ ]:
df1=pd.read_csv('yelp_data_reviewer.dat', sep='\t')
df1 = np.array(df1)
df1

In [ ]:
rev_name = []
for i in range(len(df1)):
    var=df1[i][0].split(';')
    rev_name.append(var[1:2])

In [ ]:
rev_id = []
for i in range(len(df1)):
    var=df1[i][0].split(';')
    rev_id.append(var[0:1])
rev_id=np.array(rev_id)
rev_id.shape

In [ ]:
rev_name=np.array(rev_name)
rev_name.shape

In [ ]:
rev_name_train=[]
for i in range(2908):
    temp = rev_id_train[i][0]
    for j in range (5123):
        c=0
        if(temp == rev_id[j][0]):
            rev_name_train.append(rev_name[j][0])
            c=1
            break
    if c==0:
        rev_name_train.append("Fake Name")

In [ ]:

rev_name_train = np.array(rev_name_train)
rev_name_train.shape

In [ ]:
#rev_conc=np.concatenate((rev_name_train,rev_train),axis=1)

In [ ]:
#rev_conc=np.array(rev_conc)

# Behavioral Analysis

## By Reading the given paper we draw the following conclusions
## 1. Majority fake reviews are less than 135 words

In [ ]:
rev_len=[]
for i in range(2908):
    rev_len.append(len(rev_train[i][0]))

In [ ]:
c=[]
for i in range(2908):
    count=0
    temp=rev_name_train[i][:]
    for j in range(2908):
        if (rev_name_train[i][:]==rev_name_train[j][:] and rev_date_train[i][:] ==rev_date_train[j][:] and j!=i):
            #print(i,j)
            count+=1
    #if(count>=3):
        #print(i)
    c.append(count)

In [ ]:
c=np.array(c)
c.shape

In [ ]:
c.sort()

In [ ]:
c

## Conclusion
### This shows no reviewer has made more than 5 comments on one day so we cannot verufy the analysis from yelp paper that 75% of spammers posted more than 5 reviews per day 

In [ ]:
rev_len= np.array(rev_len)

In [ ]:
rev_len.shape

In [ ]:
rev_len.sort()

In [ ]:
rev_len

In [ ]:
count_chk=0
for i in range(len(rev_len)):
    if rev_len[i] <=135:
        count_chk +=1
        

In [ ]:
count_chk

## We see that 175 reviews are less than 135 characters hence those 175 reviews have more chances to be fake reviews

# Now using this behavior

In [ ]:
from sklearn import svm
n_folds = 5
score = 0.0
skf = StratifiedKFold(rev_test, n_folds)


for train_index, test_index in skf:
    X_train, X_test = train_data_features[train_index], train_data_features[test_index]
    y_train, y_test = rev_test[train_index], rev_test[test_index]
    
    
    YPred = svm.SVC(kernel='sigmoid').fit(X_train, y_train).predict(X_test)
    YPred = np.array(YPred)
    Ylen = rev_len[test_index]
    
    sc=0
    for h in range(len(test_index)):
        if (YPred[h] != y_test[h]):
            sc+=1
    print((len(test_index)-sc)/len(test_index))
    
    for h in range(len(test_index)):
        if (YPred[h] == 'N' and Ylen[h] <135):
            #print('Shift')
            YPred[h] = 'Y'
    sc=0
    for h in range(len(test_index)):
        if (YPred[h] != y_test[h]):
            sc+=1
    print((len(test_index)-sc)/len(test_index),'Next')        

    #y.append(YPred)
    #print(svm.SVC(kernel='sigmoid').fit(X_train, y_train).score(X_test,y_test))